# Demonstration for Ohio Model User's Group

In [1]:
import os
import sys
import yaml
import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import Project
from lasso import metcouncil

### Set up the Logger

In [2]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.setLevel(logging.WARNING)

### Input/Output Locations & Parameters

In [4]:
lasso_dir = os.path.join("C:/", "Users", "david.ory", "documents", "GitHub", "Lasso")
input_dir = os.path.join(lasso_dir, "examples", "stpaul")
output_dir = os.path.join(input_dir, "ohio_mug")

stpaul_shape_file = os.path.join(input_dir, "shape.geojson")
stpaul_link_file = os.path.join(input_dir, "link.json")
stpaul_node_file = os.path.join(input_dir, "node.geojson")

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

### Data Reads

In [6]:
roadway_net = RoadwayNetwork.read(
    link_file = stpaul_link_file, 
    node_file = stpaul_node_file, 
    shape_file = stpaul_shape_file, 
    fast = True        
)

In [7]:
transit_net = TransitNetwork.read(feed_path = os.path.join(input_dir, "transit"))

In [8]:
roadway_net.links_df.columns

Index(['model_link_id', 'osm_link_id', 'shstReferenceId', 'shstGeometryId',
       'shape_id', 'u', 'v', 'A', 'B', 'locationReferences', 'distance',
       'roadway', 'name', 'ref', 'bridge', 'tunnel', 'width', 'max_speed',
       'bike_facility', 'drive_access', 'walk_access', 'bike_access',
       'truck_access', 'bus_only', 'rail_only', 'lanes', 'access', 'price',
       'trn_priority', 'ttime_assert', 'geometry'],
      dtype='object')

### Make a Scenario 

In [9]:
base_scenario = Scenario.create_scenario(base_scenario = {"road_net": roadway_net, "transit_net": transit_net})

### Make a Cube Network

In [10]:
base_scenario.road_net.links_df = base_scenario.road_net.links_df[base_scenario.road_net.links_df['drive_access'] == 1]

model_road_net = ModelRoadwayNetwork.from_RoadwayNetwork(base_scenario.road_net)
model_road_net = metcouncil.calculate_centroidconnect(model_road_net, parameters)
model_road_net.roadway_standard_to_met_council_network()

c:\users\david.ory\documents\github\lasso\lasso\roadway.py:423: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid
c:\users\david.ory\documents\github\lasso\lasso\roadway.py:288: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2021-03-16 12:21:48, WARNING: Specified variable to split: ML_lanes not in network variables: Index(['model_link_id', 'osm_link_id', 'shstReferenceId', 'shstGeometryId',
       'shape_id', 'u', 'v', 'A', 'B', 'locationReferences', 'distance',
       'roadway', 'name', 'ref', 'bridge', 'tunnel', 'width', 'max_speed',
       'bike_facility', 'drive_access', 'walk_access', 'bike_access',
       'truck_access', 'bus_only', 'rail_only', 'lanes', 'access', 'price',
       'trn_priority', 'ttime_assert', 'geometry', 'centroidconnect',
       'area_type', 'county', 'mpo', 'AADT', 'count_AM', 'count_MD',
       'count_PM', 'count_NT', 'count_daily', 'count_year', 'segment_id',
       'managed', 'trn_priority_AM', 'trn_priority_MD', 'trn_priority_PM',
       'trn_priority_NT', 'ttime_assert_AM', 'ttime_assert_MD',
       'ttime_assert_PM', 'ttime_assert_NT', 'lanes_AM', 'lanes_MD',
       'lanes_PM', 'lanes_NT'],
      dtype='object'). Returning 0.


In [11]:
model_road_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, "links.txt"),
    output_node_txt = os.path.join(output_dir, "nodes.txt"),
    output_link_header_width_txt = os.path.join(output_dir, "links_header_width.txt"),
    output_node_header_width_txt = os.path.join(output_dir, "nodes_header_width.txt"),
    output_cube_network_script = os.path.join(output_dir, "make_complete_network_from_fixed_width_file.s"),
)

In [12]:
standard_transit = StandardTransit.fromTransitNetwork(transit_net)
standard_transit.write_as_cube_lin(outpath = os.path.join(output_dir, "transit.lin") )

### Make a Project Card from a Cube Log File

In [13]:
project = Project.create_project(
        base_roadway_network = model_road_net,
        roadway_log_file = os.path.join(output_dir, "st_paul_example.log"),
        project_name = "Example",
)

2021-03-16 12:24:04, WARNING: The following attributes are specified in the changes but do not exist in the base network: ['OPERATION_final']


In [14]:
project.write_project_card(os.path.join(output_dir, "cards", "Example.yml"))

### Apply the Project Card

In [15]:
build_scenario = Scenario.create_scenario(base_scenario = {"road_net": roadway_net, "transit_net": transit_net},
                                          card_directory = os.path.join(output_dir, "cards"),
                                          validate_project_cards = False)

In [16]:
build_scenario.apply_all_projects()

C:\Users\david.ory\anaconda3\envs\lasso_x\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [17]:
build_scenario.applied_projects

['Example']

### Write Out Edited Cube Network

In [18]:
build_model_road_net = ModelRoadwayNetwork.from_RoadwayNetwork(build_scenario.road_net)
build_model_road_net.roadway_standard_to_met_council_network()

c:\users\david.ory\documents\github\lasso\lasso\roadway.py:423: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid
c:\users\david.ory\documents\github\lasso\lasso\roadway.py:288: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2021-03-16 12:26:13, WARNING: Specified variable to split: ML_lanes not in network variables: Index(['model_link_id', 'osm_link_id', 'shstReferenceId', 'shstGeometryId',
       'shape_id', 'u', 'v', 'A', 'B', 'locationReferences', 'distance',
       'roadway', 'name', 'ref', 'bridge', 'tunnel', 'width', 'max_speed',
       'bike_facility', 'drive_access', 'walk_access', 'bike_access',
       'truck_access', 'bus_only', 'rail_only', 'lanes', 'access', 'price',
       'trn_priority', 'ttime_assert', 'geometry', 'area_type', 'county',
       'mpo', 'AADT', 'count_AM', 'count_MD', 'count_PM', 'count_NT',
       'count_daily', 'count_year', 'segment_id', 'managed', 'trn_priority_AM',
       'trn_priority_MD', 'trn_priority_PM', 'trn_priority_NT',
       'ttime_assert_AM', 'ttime_assert_MD', 'ttime_assert_PM',
       'ttime_assert_NT', 'lanes_AM', 'lanes_MD', 'lanes_PM', 'lanes_NT'],
      dtype='object'). Returning 0.


In [19]:
build_model_road_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, "links.txt"),
    output_node_txt = os.path.join(output_dir, "nodes.txt"),
    output_link_header_width_txt = os.path.join(output_dir, "links_header_width.txt"),
    output_node_header_width_txt = os.path.join(output_dir, "nodes_header_width.txt"),
    output_cube_network_script = os.path.join(output_dir, "make_complete_network_from_fixed_width_file.s"),
)